## CLUSTERING : DATA WRANGLING WITH ZILLOW & THE MALL

25 January 2023

In [1]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import prepare
from env import sql_connexion

from sklearn.preprocessing import MinMaxScaler

In [2]:
# # getting sql_connexion from acquire.py

# url = sql_connexion('zillow')

In [3]:
# # defining the query

# query = '''
#         select *
#         from properties_2017
#         join predictions_2017 using (parcelid)
#         join propertylandusetype USING(propertylandusetypeid)
#         '''


In [4]:
# putting it together 

df = pd.read_csv('zillow.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77579 non-null  int64  
 1   typeconstructiontypeid        223 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77579 non-null  float64
 4   heatingorsystemtypeid         49571 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      207 non-null    float64
 7   airconditioningtypeid         25007 non-null  float64
 8   id                            77579 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77579 non-null  float64
 11  bedroomcnt                    77579 non-null  float64
 12  buildingqualitytypeid         49809 non-null  float64
 13  c

/var/folders/v1/7y2hc6q17wl2rjlpg8h9g78h0000gn/T/ipykernel_2123/3709608481.py:3: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('zillow.csv')


In [5]:
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,4,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [6]:
# sorting by transaction date

df.sort_values('transactiondate', inplace = True)

In [7]:
# keeping only the unique parcel ids

df = df.drop_duplicates(subset = 'parcelid', keep = 'last')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 0 to 77578
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77381 non-null  int64  
 1   typeconstructiontypeid        222 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77381 non-null  float64
 4   heatingorsystemtypeid         49440 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      206 non-null    float64
 7   airconditioningtypeid         24953 non-null  float64
 8   id                            77381 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77381 non-null  float64
 11  bedroomcnt                    77381 non-null  float64
 12  buildingqualitytypeid         49672 non-null  float64
 13  c

In [8]:
#describing the data

df.describe().T

,count,mean,std,min,25%,50%,75%,max
parcelid,77381.0,1.300715e+07,3.481346e+06,1.071186e+07,1.153830e+07,1.253157e+07,1.421183e+07,1.676893e+08
typeconstructiontypeid,222.0,6.040541e+00,5.572847e-01,4.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,1.300000e+01
storytypeid,50.0,7.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
propertylandusetypeid,77381.0,2.618263e+02,5.141199e+00,3.100000e+01,2.610000e+02,2.610000e+02,2.660000e+02,2.750000e+02
heatingorsystemtypeid,49440.0,3.920510e+00,3.592779e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,2.400000e+01
buildingclasstypeid,15.0,3.933333e+00,2.581989e-01,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
architecturalstyletypeid,206.0,7.388350e+00,2.734542e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
airconditioningtypeid,24953.0,1.813289e+00,2.967894e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
id,77381.0,1.495139e+06,8.609071e+05,3.490000e+02,7.520700e+05,1.497932e+06,2.240535e+06,2.982274e+06
basementsqft,50.0,6.797200e+02,6.897035e+02,3.800000e+01,2.730000e+02,5.150000e+02,7.965000e+02,3.560000e+03


In [9]:
# shape of df

df.shape

(77381, 69)

In [10]:
# some distributions : year built

df['yearbuilt'].min(), df['yearbuilt'].max()

(1824.0, 2016.0)

In [11]:
# some distributions : number of fireplaces

df['fireplacecnt'].max(), df['fireplacecnt'].min()

(5.0, 1.0)

In [12]:
## some distributions : number of fireplaces

df['basementsqft'].max(), df['basementsqft'].min()

(3560.0, 38.0)

In [13]:
df.drop(columns = ['id.1', 'parcelid'], inplace = True)

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute.   
Run the function and document takeaways from this on how you want to handle missing values.

In [14]:
len(df)

77381

In [15]:
# putting all the columns into a list

all_col = ['typeconstructiontypeid', 'storytypeid', 'propertylandusetypeid',
           'heatingorsystemtypeid','buildingclasstypeid', 'airconditioningtypeid',
           'id', 'basementsqft', 'bathroomcnt', 'bedroomcnt','buildingqualitytypeid',
           'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet',
           'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 
           'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
           'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
           'garagecarcnt','garagetotalsqft', 'hashottuborspa', 'latitude',
           'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
           'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'propertycountylandusecode',
           'fireplaceflag', 'structuretaxvaluedollarcnt','propertyzoningdesc',
           'rawcensustractandblock', 'regionidcity', 'regionidcounty', 
           'regionidneighborhood','regionidzip', 'roomcnt','threequarterbathnbr', 
           'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
           'numberofstories', 'taxvaluedollarcnt', 'assessmentyear', 
           'landtaxvaluedollarcnt', 'taxamount', 'taxdelinquencyyear', 
           'censustractandblock', 'logerror']


In [16]:
# # converting 0, blanks, nulls to nan

# df = df.replace([0, ' ', 'NULL'], np.nan)

# df.head(1)

In [17]:
# # creating df of rows with null values

# missing_rows = df.isna()
# missing_rows.head(1)

In [18]:
# # assuring that missing_rows is boolean
# missing_rows.dtypes


In [19]:
# # finding sum of each row

# missing_num_rows = missing_rows.sum()
# missing_num_rows

In [20]:
# # finding the percentage of missing rows

# pc_missing_rows = round((missing_num_rows / len(df)), 2)
# pc_missing_rows

In [21]:
# # alternate to the above

# dnd = (df.isna().mean().round(4))
# dnd

In [22]:
# # creating dataframe with the missing row-numbers
# missing = pd.DataFrame(missing_num_rows)
# missing

In [23]:
# # adding the pc to the missing df

# missing['pc_missing'] = pc_missing_rows
# missing

In [24]:
# # renaming the 0 column

# missing = missing.rename(columns = {0 : 'num_missing_rows'})
# missing

In [25]:
# # sorting by pc_missing date

# missing.sort_values('pc_missing', ascending = False, inplace = True)
# missing


In [26]:
# in-class example 

def null_counter(df):
    
    num_coluns = ['name', 'num_rows_missing', 'pc_rows_missing']
    new_df = pd.DataFrame(columns = num_coluns)
    
    for col in list(df.columns):
        num_missing = df[col].isna().sum()
        
        pc_missing = num_missing / df.shape[0]
        
        add_df = pd.DataFrame([{'name' : col, 'num_rows_missing': num_missing, 'pc_rows_missing' : pc_missing}])
        
        new_df = pd.concat([new_df, add_df], axis = 0)
        
    new_df.set_index('name', inplace = True)
    
    return new_df

In [27]:
# in-class example 
null_df = null_counter(df)

null_df.sort_values(by = 'pc_rows_missing', ascending = False)

,num_rows_missing,pc_rows_missing
name,,
buildingclassdesc,77366,0.999806
buildingclasstypeid,77366,0.999806
finishedsquarefeet13,77340,0.999470
storydesc,77331,0.999354
basementsqft,77331,0.999354
storytypeid,77331,0.999354
yardbuildingsqft26,77311,0.999095
fireplaceflag,77209,0.997777
architecturalstyletypeid,77175,0.997338


In [28]:
# function to do the above 
# null_counter

def missing_rows(df):
    
    # converting 0, blanks, nulls to nan
    df = df.replace([' ', 'NULL'], np.nan)

    # creating df of rows with null values
    missing_rows = df.isna()
    
    # finding sum of each row
    missing_num_rows = missing_rows.sum()
    
    # finding the percentage of missing rows
    pc_missing_rows = (df.isna().mean().round(4))
    
    # creating dataframe with the missing row-numbers
    df = pd.DataFrame(missing_num_rows)
    
    # adding the pc to the missing df
    df['pc_missing'] = pc_missing_rows

    # renaming the 0 column
    df = df.rename(columns = {0 : 'num_missing_rows'})
    
    # sorting to have lowest pc missing at the top
    df.sort_values('pc_missing', ascending = False, inplace = True)
    
    return df

In [29]:
missing = missing_rows(df)
missing.head()

,num_missing_rows,pc_missing
buildingclassdesc,77366,0.9998
buildingclasstypeid,77366,0.9998
finishedsquarefeet13,77340,0.9995
storydesc,77331,0.9994
basementsqft,77331,0.9994


## Prepare

### 1 . Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.


In [30]:
# looking at the columns

df.columns

Index(['typeconstructiontypeid', 'storytypeid', 'propertylandusetypeid',
       'heatingorsystemtypeid', 'buildingclasstypeid',
       'architecturalstyletypeid', 'airconditioningtypeid', 'id',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
       'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildi

In [31]:
# looking at prperty types

df['propertylandusedesc'].value_counts()

Single Family Residential                     52320
Condominium                                   19294
Duplex (2 Units, Any Combination)              2009
Planned Unit Development                       1944
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              535
Cluster Home                                    333
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              37
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

In [32]:
#keeping only units of Single Family Residential 

# df = df[df['propertylandusedesc'] == 'Single Family Residential']

Decided against SFR as the separator, bc many SFRs have +1 unit.

In [33]:
# finding how many properties have how many units

df['unitcnt'].value_counts()

1.0      47293
2.0       2018
4.0        723
3.0        525
45.0         1
42.0         1
6.0          1
237.0        1
Name: unitcnt, dtype: int64

In [34]:
# keeping only properties with 1 unit

df = df[df['unitcnt'] == 1]
df.shape

(47293, 67)

## 2 . Create a function that will drop rows or columns based on the percent of values that are missing : 
```handle_missing_values(df, prop_required_column, prop_required_row)```.


    -The input:
        A dataframe  
        
        A number between 0 and 1 that represents the proportion of rows for each column with non-missing values This number is the amount required to keep the column. For example, if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).  
        
        A number between 0 and 1 that represents the proportion of columns/variables for each row with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).  
        
    -The output:
        The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.  
        
    -hint:
        Look up the dropna documentation.
        You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.


### alternate nº3

In [35]:
# alternate function by Bill

def drops(df, thresh1, thresh2):
    
    # dropping if nulls more than a certain amount
    # th1 = rows, th2 = col
    
    loop = list(range(0, len(df.axes[1])))
    adf = []
    
    # chooses column from 0 to x
    for i in loop: 
        x = df.iloc[:,i].isna().sum()
        
        #appending to above blank list
        adf.append(x)
        
    fjf = []
    for i in adf:
        x = i / len(df.axes[0])
        # diving nulls by num rows to get a pc
        fjf.append(x)
        
    # make lists into arrays
    indices = np.array(fjf)
    indices2 = np.where(indices > thresh1)[0]
    
    # array of indices of columsn to drop
    df = df.drop(df.columns[indices2], axis = 1)
    
    ddd = []
    x = df.isnull().sum(axis = 1)
    for i in x:
        # going into the rows, counting rows for each col
        yo = i / len(x) # gets pc of nulls in row
        
        ddd.append(yo)
    indices3 = np.array(ddd)
    indices4 = np.where(indices3 > thresh2)[0]
    
    # dropping indices4
    df = df.drop(indices4)
    
    return df

In [36]:
df1 = drops(df, 0.6, 0.6)
df1.head(1)

,propertylandusetypeid,heatingorsystemtypeid,airconditioningtypeid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,airconditioningdesc,heatingorsystemdesc,propertylandusedesc
3,261.0,2.0,NaN,2288172,3.0,4.0,8.0,3.0,2376.0,2376.0,...,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.10341,2017-01-01,NaN,Central,Single Family Residential


In [37]:
# # giving the index a name

# missing.index.names = ['descriptor']


# # reset index

# missing.reset_index(inplace = True)
# missing.head(1)

In [38]:
# # looking at rows with less than 55% missing values
# missing[missing['pc_missing'] < 0.55].head(3)

In [39]:

# missing.iloc[0]
    
#     missing.loc[(missing['index'] == col) &
#                 (missing['pc_missing'] < 0.6)]

In [40]:
# # making a function to access each row

# def thing(df):
#     for col in all_col:

#         # accessing each row in the index
#         df = df.loc[(df['descriptor'] == col) &
#                    (np.where(df['pc_missing'] < 0.55, 1, 0))]

#     return df

In [41]:
# thing(df)

In [42]:
# np.where(missing['pc_missing'] < 0.55, 1, 0)

In [43]:
# for col in all_col:
#     df = df.loc[(df['descriptor'] == col)]
#     df

In [44]:
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN
6,NaN,NaN,261.0,2.0,NaN,NaN,1.0,781532,NaN,3.0,...,6.037461e+13,-0.001011,2017-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
7,NaN,NaN,261.0,NaN,NaN,NaN,NaN,870991,NaN,1.0,...,6.037302e+13,0.101723,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
70,NaN,NaN,261.0,7.0,NaN,NaN,NaN,1061201,NaN,1.0,...,6.037110e+13,0.021262,2017-01-02,NaN,NaN,NaN,Floor/Wall,Single Family Residential,NaN,NaN


In [45]:
# # combining the two dfs

# #making list of df and missing
# frames = [df, missing]

# # combining into new df
# new = pd.concat(frames)

# new.drop(columns = ['descriptor'], inplace = True)

# display(new)
# df.shape, new.shape, missing.shape

In [46]:
# new2 = pd.concat([df, missing], axis = 1, join = 'outer')


Returns 47323 rows × 70 columns.

In [47]:
# df = df.T
# df

In [48]:
# # making list of df and missing
# frames = [df, missing]

# # missing.drop(columns = ['descriptor'], inplace = True)

# # combining into new df
# new = pd.concat(frames, axis = 0, join = 'outer')

# new


In [49]:
# df.shape, new.shape, missing.shape

dfo = df.copy()

In [50]:
dfo = dfo.append(missing, ignore_index=True)
dfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47360 entries, 0 to 47359
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        0 non-null      float64
 1   storytypeid                   0 non-null      float64
 2   propertylandusetypeid         47293 non-null  float64
 3   heatingorsystemtypeid         46565 non-null  float64
 4   buildingclasstypeid           8 non-null      float64
 5   architecturalstyletypeid      0 non-null      float64
 6   airconditioningtypeid         22020 non-null  float64
 7   id                            47293 non-null  float64
 8   basementsqft                  0 non-null      float64
 9   bathroomcnt                   47293 non-null  float64
 10  bedroomcnt                    47293 non-null  float64
 11  buildingqualitytypeid         46803 non-null  float64
 12  calculatedbathnbr             47247 non-null  float64
 13  d

/var/folders/v1/7y2hc6q17wl2rjlpg8h9g78h0000gn/T/ipykernel_2123/971790651.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfo = dfo.append(missing, ignore_index=True)


In [51]:
missing.head()

,num_missing_rows,pc_missing
buildingclassdesc,77366,0.9998
buildingclasstypeid,77366,0.9998
finishedsquarefeet13,77340,0.9995
storydesc,77331,0.9994
basementsqft,77331,0.9994


In [52]:
missing[missing['pc_missing'] > 0.55]

,num_missing_rows,pc_missing
buildingclassdesc,77366,0.9998
buildingclasstypeid,77366,0.9998
finishedsquarefeet13,77340,0.9995
storydesc,77331,0.9994
basementsqft,77331,0.9994
storytypeid,77331,0.9994
yardbuildingsqft26,77311,0.9991
fireplaceflag,77209,0.9978
architecturalstyletypeid,77175,0.9973
architecturalstyledesc,77175,0.9973


In [53]:
# this is the way to 

ciao = missing[missing['pc_missing'] > 0.55].index.to_list()
df.drop(columns = ciao, inplace = True)
df.shape

(47293, 33)

In [54]:
# counting how many nans are left

df.isna().sum()

propertylandusetypeid              0
heatingorsystemtypeid            728
id                                 0
bathroomcnt                        0
bedroomcnt                         0
buildingqualitytypeid            490
calculatedbathnbr                 46
calculatedfinishedsquarefeet       7
finishedsquarefeet12              47
fips                               0
fullbathcnt                       46
latitude                           0
longitude                          0
lotsizesquarefeet               1235
propertycountylandusecode          0
propertyzoningdesc               230
rawcensustractandblock             0
regionidcity                    1041
regionidcounty                     0
regionidzip                       20
roomcnt                            0
unitcnt                            0
yearbuilt                         14
structuretaxvaluedollarcnt        63
taxvaluedollarcnt                  0
assessmentyear                     0
landtaxvaluedollarcnt              0
t

In [55]:
# missing[missing['pc_missing'] > 0.55]

In [56]:
# drops a row if all that it has are NaNs

# df.dropna(thresh = 27, how = 'any', axis = 0)

# df.dropna(thresh = number of remaining cols * entered_number (0.844), axis = 0)

In [5]:
# function

def handle_missing_values(df, prop_required_column, prop_required_row):
    
    # copy of original df
    df2 = df.copy()
    
    # list of all columns with +55pc missing values
    ciao = missing[missing['pc_missing'] > prop_required_column].index.to_list()
    
    # dropping 
    #dfo.dropna(columns = ciao)
    
    # dropping rows with pc missing values
    dfo.dropna(thresh = (dfo.shape[0] * prop_required_row), how = 'any', axis = 0)
    
    return dfo


In [7]:
handle_missing_values(df, 0.8, 0.6)

NameError: name 'df' is not defined

In [1]:
dfo.isnull().sum()

NameError: name 'dfo' is not defined

In [ ]:
#df.dropna(thresh = (len(df) * .8), axis = 0)
 
    # this kills the kernel
    
    
th = len(df) * 0.84
    
#df.dropna(th)

In [ ]:
th

In [ ]:
thresh

In [ ]:
df.dropna(thresh= 27)

### 4 or some num : Mall customers

In [ ]:
# Mall customers

mall_query = '''
            SELECT * 
            FROM customers
            '''

In [ ]:
mall_url = sql_connexion('mall_customers')

In [ ]:
mall_df = pd.read_sql(mall_query, mall_url)

In [ ]:
mall_df.head(2)

In [ ]:
# making histograms with the dataframe

col = mall_df.columns.to_list()

for col in mall_df:
    plt.hist(mall_df[col])
    print(f'{col} distro')
    plt.show()


    1. Acquire data from the customers table in the mall_customers database.
    2. Summarize the data (include distributions and descriptive statistics).
    3. Detect outliers using IQR.
    4. Split data into train, validate, and test.
    5. Encode categorical columns using a one hot encoder (pd.get_dummies).
    6. Handles missing values.
    7. Scaling


In [ ]:
mall_df['age'].quantile([0.25, 0.75])

In [ ]:
q1, q3 = np.percentile(mall_df['age'], [25, 75])
q1, q3 

In [ ]:
def outlier_bound_calc(df, variable):
    
    q1, q3 = np.percentile(df[variable], [25, 75])
    
    iqr_val = q3 - q1
    
    lower_bound = q1 - (1.5 * iqr_val)

    upper_bound = q3 + (1.5 - iqr_val)
    
    return print(f'For {variable}, the lower bound is {lower_bound}, and the upper bound is {upper_bound}.')

In [ ]:
# reutrning the above function with a dataframe

age_outliers = outlier_bound_calc(mall_df, 'age')

In [ ]:
# isolating numeric & integers ; getting rid of objects
num_df = mall_df.select_dtypes(exclude = object)
num_df.info()

In [ ]:
# finding the discretes / keeping only the objects

inc_obj = mall_df.select_dtypes(include = object)
inc_obj.info()

In [ ]:
# fucntion

cols = num_df.columns.to_list()

for col in cols:
    outlier_bound_calc(num_df, [col])

In [ ]:
# train, test, split

prepare.my_train_test_split(mall_df, 'spending_score')

In [ ]:
train, validate, test

In [ ]:
X_train.head()

In [ ]:
# make dumies

dumdf = pd.get_dummies(train)
dumdf

In [ ]:
# function to create dummies for a generic df

def encoding(df, cols, drop_first = True):
    
    for col in cols:
        dummies = pd.get_dummies(df[f'{col}'], drop_first = drop_first)
        df = pd.concat([df, dummies], axis = 1)
    return df



In [ ]:
train_encoded = encoding(train, inc_obj, drop_first = True)

In [ ]:
# handle missing
mall_df.isna().sum()

In [ ]:
# scalling of age & income

mms = MinMaxScaler()

train_encoded[['age', 'annual_income']] = mms.fit_transform(train_encoded[['age', 'annual_income']])........

### pd.iterrows https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html